In [1]:
import sqlite3
import pandas as pd
import csv
import numpy as np

In [20]:
conn = sqlite3.connect('text_db.db')
cursor = conn.cursor()


In [21]:
cursor.execute('''CREATE TABLE IF NOT EXISTS text_db (
                    text TEXT,
                    embeding INTEGER

                )''')

In [22]:
df = pd.read_excel('out1.xlsx', names = ['Sentences', 'Embbeding'])

In [23]:
df.to_csv('out1.csv', index = False)

In [24]:
with open('out1.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Пропуск заголовков столбцов
    cursor.executemany('INSERT INTO text_db VALUES (?, ?)', csv_data)

In [25]:
conn.commit()
conn.close()

In [26]:
conn = sqlite3.connect('text_db.db')
cursor = conn.cursor()

In [27]:
cursor.execute("SELECT * FROM text_db")
rows = cursor.fetchall()

In [39]:
input_vector = [float(x) for x in rows[0][1].split(',')]

In [105]:
# vectors = [np.frombuffer(vector[0]) for vector in vectors]

In [41]:
from scipy.spatial.distance import cosine

In [65]:
!pip show sqlite3

In [45]:
# Расчет косинусного расстояния и сохранение результатов
distances = []
for row in rows:
    vector = [float(x) for x in row[1].split(',')]
    distance = cosine(input_vector, vector)
    distances.append((distance, row[1], row[0]))  # Замените на соответствующий столбец с другими данными

# Сортировка результатов по возрастанию расстояния
distances.sort(key=lambda x: x[0])

# Вывод 10 самых близких расстояний и соответствующих данных
# for i in range(10):
#     print(f"Расстояние: {distances[i][0]}, Другие данные: {distances[i][1]}")

# Закрытие соединения с базой данных
conn.close()

In [57]:
out = []
for i in range(10):
    out.append(distances[i][2])

In [ ]:
После создания базы данных, вы можете использовать ее внутри Docker контейнера. Для этого вам нужно создать Dockerfile, который будет содержать инструкции для создания контейнера. Вот пример Dockerfile:

```Dockerfile
FROM python:3.9

# Копирование файлов в контейнер
COPY database.db /app/database.db
COPY script.py /app/script.py
COPY data.csv /app/data.csv

# Установка зависимостей
RUN pip install sqlite3

# Запуск скрипта при запуске контейнера
CMD python /app/script.py
```

Затем, вы можете собрать Docker образ и запустить контейнер с помощью следующих команд:

```
docker build -t my_app .
docker run my_app
```

При этом, файлы `database.db`, `script.py` и `data.csv` должны находиться в той же директории, где находится Dockerfile.